In [2]:
import re
from decimal import Decimal
import torch
import torch.nn.functional as F

def parse_serialized_tensor(serialized: str) -> torch.Tensor:
    """
    Parses a serialized tensor string of the form:
      (shape=(2, 2), data=[-1.0, 2.5, -3.6, 4.7])
    and returns a torch.Tensor instance.

    This version tolerates extra whitespace between symbols, e.g.:
      ( shape = (2 , 2 ) , data = [ -1.0 , 2.5 , -3.6 , 4.7 ] )
    """
    # Regex patterns that allow for arbitrary spaces around 'shape=', 'data=', etc.
    shape_pattern = r"shape\s*=\s*\(\s*([^)]*)\)"
    data_pattern = r"data\s*=\s*\[\s*([^]]*)\]"
    
    # Search for 'shape=...' and 'data=...' in the input
    shape_match = re.search(shape_pattern, serialized)
    data_match = re.search(data_pattern, serialized)
    
    if not shape_match or not data_match:
        raise ValueError("Invalid serialized tensor format. Could not find shape or data.")
    
    # Extract the substring that represents the shape, e.g. "2, 2"
    shape_str = shape_match.group(1).strip()
    # Split by commas and convert each dimension to int
    shape = tuple(
        int(dim.strip()) 
        for dim in shape_str.split(',') 
        if dim.strip()  # ignore empty pieces if extra commas/spaces
    )
    
    # Extract the substring for the data, e.g. "-1.0, 2.5, -3.6, 4.7"
    data_str = data_match.group(1).strip()
    # Convert each piece into a Decimal, then to float
    data_list = [
        Decimal(value.strip())
        for value in data_str.split(',')
        if value.strip()
    ]
    data_floats = [float(x) for x in data_list]
    
    # Construct the tensor and reshape accordingly
    tensor = torch.tensor(data_floats, dtype=torch.float).reshape(shape)
    return tensor


In [3]:
serialized_str = "(shape = (2 ,2), data = [ -1.0 , 2.5 , -3.6 , 4.7 ])"
t = parse_serialized_tensor(serialized_str)
print(t)
print(t.shape)  # torch.Size([2, 2])

tensor([[-1.0000,  2.5000],
        [-3.6000,  4.7000]])
torch.Size([2, 2])


In [4]:
a = torch.nn.CELU(alpha=-0.001)(torch.zeros(2,3))
print(a)

tensor([[-0., -0., -0.],
        [-0., -0., -0.]])


In [5]:
x = torch.zeros(2,3,4) - 60.0
x

tensor([[[-60., -60., -60., -60.],
         [-60., -60., -60., -60.],
         [-60., -60., -60., -60.]],

        [[-60., -60., -60., -60.],
         [-60., -60., -60., -60.],
         [-60., -60., -60., -60.]]])

In [6]:
F.rrelu(x,lower=-10.0,upper=1/3)

tensor([[[290., 290., 290., 290.],
         [290., 290., 290., 290.],
         [290., 290., 290., 290.]],

        [[290., 290., 290., 290.],
         [290., 290., 290., 290.],
         [290., 290., 290., 290.]]])

In [7]:
rrelu = torch.nn.RReLU(lower=-10.0)
rrelu.train(mode=False)

RReLU(lower=-10.0, upper=0.3333333333333333)

In [8]:
rrelu(x)

tensor([[[290., 290., 290., 290.],
         [290., 290., 290., 290.],
         [290., 290., 290., 290.]],

        [[290., 290., 290., 290.],
         [290., 290., 290., 290.],
         [290., 290., 290., 290.]]])

In [15]:
a = torch.arange(6,dtype=torch.float32).reshape(2,3)
a

tensor([[0., 1., 2.],
        [3., 4., 5.]])

In [16]:
a.sum(0)

tensor([3., 5., 7.])

In [17]:
a.sum(1)

tensor([ 3., 12.])

In [18]:
torch.mean(a,0)

tensor([1.5000, 2.5000, 3.5000])

In [19]:
torch.mean(a,1)

tensor([1., 4.])